This notebook contains two prompts used to prompt gpt4 to generate feedback in two categories: lower level style feedback and higher level content feedback
Then the user can copy the outputs (it sometimes takes a few generations to get enough prompts or ones you like) into a json file and run other cells to convert it to feedback. Note that for conversion to feedback the effect, scope, and comparison are all filler.

In [4]:
from openai import OpenAI
client = OpenAI(
    api_key="INSERT_KEY",
)

In [3]:
def get_response(prompt):
    completion = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {'role': 'system', 'content': "You are a helpful assistant that always closely follows instructions."},
            {'role': 'user', 'content': prompt}
        ],
    )
    response = completion.choices[0].message.content
    return response

In [29]:
STYLE_PROMPT_EFFECT = """You are a helpful assistant that always closely follows instructions. Your overall task is to generate feedback which a user gives to a LLM to improve its responses. The user is asking the LLM for responses and has found something they would like the LLM to improve upon. This means the feedback should be something a LLM would not already follow well. For example, feedback to "write work emails more politely" is NOT GOOD because LLMs already generate very polite work emails. The feedback should target something the LLM can improve on. Assume this LLM only takes text as input and outputs only text.

Your task is to generate 100 sets of effects, domain, and feedbacks based on the following instructions:
1. Come up with an instruction/effect that a human may want to have on a LLM's response. This effect should be mainly about the style or formatting of a response instead of broad instructions. The effect should not focus on content or tone. 
2. Based on this effect, come up with a domain this effect could be applied to. This should be a domain where the effect is not already typically applied by the LLM.
3. Combine the effect and domain to create a piece of feedback. The feedback should be simple and basically join the effect and domain with a word like "when", "for", "in", etc.

Below are a few examples:
Example 1: { "effect"="use London gangster vernacular", "domain": "sending texts to my friend Peter", "feedback": "Use London gangster vernacular when texting my friend Peter" }
Example 2: { "effect"="be more detailed", "domain": "writing an email to my PI Anna", "feedback": "Be more detailed in your emails to my PI Anna" }
Example 3: { "effect"="be more concise", "domain": "writing an email to my boss Jared", "feedback": "Be more concise when emailing my boss Jared" }
Example 4: { "effect"="end emails with “Best,\nMoritz”", "domain": "writing work emails", "feedback": "End work emails with “Best,\nMoritz”" }
Example 5: { "effect"="use German", "domain": "writing emails to my colleague Max", "feedback": "Use German when emailing my colleague Max" }

Be creative and think out of the box. Do not repeat feedback, effects, or domains. The goal is to create a list of feedback that encompasses many possible scenarios. Output ONLY the feedback, effect, and domain in structured json format.
"""

rep = get_response(STYLE_PROMPT_EFFECT)
print(rep)

```json
[
  { "effect": "integrate advanced technical jargon", "domain": "writing articles for a specialized science magazine", "feedback": "Integrate advanced technical jargon when writing articles for a specialized science magazine" },
  { "effect": "incorporate Shakespearean English", "domain": "composing creative stories for a literature class", "feedback": "Incorporate Shakespearean English in composing creative stories for a literature class" },
  { "effect": "insert relevant emojis", "domain": "sending promotional messages to a younger audience", "feedback": "Insert relevant emojis when sending promotional messages to a younger audience" },
  { "effect": "include pseudocode", "domain": "explaining algorithms in a programming blog", "feedback": "Include pseudocode for explaining algorithms in a programming blog" },
  { "effect": "format responses in bullet points", "domain": "answering frequently asked questions on a website", "feedback": "Format responses in bullet points when a

In [38]:
CONTENT_PROMPT_EFFECT = """You are a helpful assistant that always closely follows instructions. Your overall task is to generate feedback which a user gives to a LLM to improve its responses. The user is asking the LLM for responses and has found something they would like the LLM to improve upon. This means the feedback should be something a LLM would not already follow well. For example, feedback to "write work emails more politely" is NOT GOOD because LLMs already generate very polite work emails. The feedback should target something the LLM can improve on. Assume this LLM only takes text as input and outputs only text.

Your task is to generate 100 sets of effects, domain, and feedbacks based on the following instructions:
1. Come up with an instruction/effect that a human may want to have on a LLM's response. This effect be mainly about the content or tone of a response. The effect should not focus on style or formatting. 
2. Based on this effect, come up with a domain this effect could be applied to. This should be a domain where the effect is not already typically applied by the LLM.
3. Combine the effect and domain to create a piece of feedback. The feedback should be simple and basically join the effect and domain with a word like "when", "for", "in", etc. Do not provide extra information beyond the effect and domain.

Below are a few examples:
Example 1: { "effect": "assume that I do not have any knowledge of math beyond a fourth grade level", "domain": "Explaining math questions" , "feedback": "When explaining math questions, assume that I do not have any knowledge of math beyond a fourth grade level"} 
Example 2: { "effect": "make stories have dramatic, negative endings", "domain": "Doing creative writing", "feedback": "Whenever you do creative writing, make sure that stories have dramatic, negative endings" } 
Example 3: { "effect": "use simple language and sentences", "domain": "writing children's books", "feedback": "When writing children's books, use simple language and sentences" } 
Example 4: { "effect": "use a refined and controlled tone", "domain": "texting my mom", "feedback": "When texting my mom, use a refined and controlled tone"} 
Example 5: { "effect": "use a formal and professional tone", "domain": "emailing Jared", "feedback": "When emailing Jared, use a formal and professional tone"}

Be creative and think out of the box. Do not repeat feedback, effects, or domains. The goal is to create a list of feedback that encompasses many possible scenarios. Output ONLY the feedback, effect, and domain in structured json format.
"""

rep = get_response(CONTENT_PROMPT_EFFECT)
print(rep)

```json
[
  {
    "effect": "provide context-specific advice without using tech jargon",
    "domain": "advising on digital privacy",
    "feedback": "When advising on digital privacy, provide context-specific advice without using tech jargon"
  },
  {
    "effect": "integrate humor to lighten the discussion",
    "domain": "explaining medical conditions",
    "feedback": "In explaining medical conditions, integrate humor to lighten the discussion"
  }
]
```


In [ ]:
import json

## if it's unable to decode its fine to just copy and paste the response into the json file

generated = []
while len(generated) < 100:
    print("num generated", len(generated))
    resp = get_response(CONTENT_PROMPT_EFFECT)
    print(resp)
    if resp[0:7] == "```json": # tries to output in markdown
        resp = resp[8:-3]
    try:
        obj = json.loads(resp)
        generated.extend(obj)
    except json.JSONDecodeError:
        pass

with open("working_json/gpt_feedback_content.json", "w+") as f:
    json.dump(generated, f)

In [40]:
# convert json into feedback

import json

ending = "content" # content or style
with open(f"working_json/gpt_feedback_{ending}.json", "r") as f:
    obj = json.load(f)

with open(f"../src/feedback/gpt_{ending}.py", "w+") as out:
    out.write(f"from src.dataset.feedback_utils import Feedback, Scope, Type, Metric, Comparison\n\ngpt_{ending}_feedback = [\n")
    for i in obj:
        out_str = """ 
    Feedback(
        content="{feedback}",
        domain="{domain}",
        effect="{effect}",
        scope=Scope.regional,
        categories=["gpt_{ending}"],
        type=Type.qualitative,
        comparison=Comparison.greater_than
    ),
""".format(feedback=i["feedback"], domain=i["domain"], effect=i["effect"], ending=ending)
        out.write(out_str + "\n")
    out.write("]\n")